# SAE31 – Projet FluxMedia

Ce projet s'inscrit dans le cadre de la SAE 31 et a pour objectif d'explorer l'utilisation de GStreamer et les fonctionnalités du module Jupyter Widgets pour la manipulation et la visualisation de flux audio au sein d'un notebook Jupyter.

Le travail est divisé en deux parties :

    Niveau Bart :
        L’objectif de ce projet était de concevoir une interface graphique minimale directement intégrée dans le 
        notebook Jupyter. Cette interface devait permettre à l’utilisateur d’interagir avec des pipelines 
        GStreamer en utilisant le module ipywidgets.

        Elle devait offrir la possibilité de sélectionner et d’écouter des fichiers audio aux formats FLAC, WAV 
        et MP3, tout en affichant des métadonnées, la pochette d’album si disponible, ainsi qu’une représentation 
        visuelle du son.

        Enfin, l’interface devait être compatible avec les fichiers fournis sur Moodle, tout en prenant en charge 
        d’autres fichiers audio externes.
 

    Niveau Lisa :
        Pour le niveau Lisa, nous avions pour objectif d’améliorer l’interface crée au niveau Bart en ajoutant 
        des fonctionnalités pertinentes.
        Après une étude des fonctionnalités exploitables sur GStreamer, nous avons décidé de sélectionner les 
        suivantes :
            -Afficher un spectrogramme
            -Possibilité d’augmenter/diminuer le volume d’un son
            -Possibilité d’accélérer/ralentir la vitesse d’un son
            -Conversion de fichiers aux formats FLAC, WAV et MP3
 





Ce travail a été réalisé par URSO Jean-Pascal et MELLANO Lancelot.



In [3]:
import ipywidgets as widgets




# Fonction principale pour exécuter les différentes actions
def execute_bouton(changement):
    if not fichier_televerse.value:
        print("Veuillez téléverser un fichier.")
        return

    uploaded_file = fichier_televerse.value[0]
    file_name = uploaded_file['name']

        
    # Niveau Bart :    
        
    # Affichage de la pochette + son
    if checkbox_pochette.value :
        nom_base = file_name.rsplit('.', 1)[0]  # Sépare le nom du fichier et son extension en partant de la fin grâce à cela "('.', 1)", avec le [0] on récupère que la première parti de la liste qui correspond au nom du fichier sans l'extension
        fichier_jpg = f"{nom_base}.jpg"  # On ajoute ".jpg" au nom récupéré pour obtenir le fichier image associé au son.
        !gst-launch-1.0 filesrc location={file_name} ! decodebin ! audioconvert ! queue ! autoaudiosink filesrc location={fichier_jpg} ! jpegdec ! imagefreeze ! queue ! autovideosink
        
        
    # Affichage des métadonnées + son
    if checkbox_metadonnes.value :
        nom_base = file_name.rsplit('.', 1)[0]
        fichier_srt = f"{nom_base}.srt"
        !gst-launch-1.0 filesrc location={file_name} ! decodebin ! audioconvert ! queue ! autoaudiosink videotestsrc pattern=black ! textoverlay name=overlay ! videoconvert ! queue ! autovideosink filesrc location={fichier_srt} ! subparse ! overlay.
       

    # Représentation visuelle + son
    if checkbox_rps_visu.value :
        !gst-launch-1.0 filesrc location={file_name} ! decodebin ! audioconvert ! tee name=piste piste. ! wavescope style=3 ! videoconvert ! queue ! autovideosink piste. ! queue ! autoaudiosink
        
        
    # Niveau Lisa :    
        

    # Spectrogramme + son
    if checkbox_rps_spec.value :
        !gst-launch-1.0 filesrc location={file_name} ! decodebin ! audioconvert ! tee name=t t. ! queue ! spectrascope shader=3 shade-amount=65280 ! videoconvert ! autovideosink t. ! queue ! autoaudiosink
        
    # Réglage du volume
    if checkbox_volume.value:
        volume_value = volume_slider.value
        !gst-launch-1.0 filesrc location={file_name} ! decodebin ! audioconvert ! volume volume={volume_value} ! autoaudiosink
    
    # Réglage de la vitesse lent/rapide
    if checkbox_vitesse.value:
        vitesse_value = vitesse_slider.value
        !gst-launch-1.0 filesrc location={file_name} ! decodebin ! audioconvert ! pitch tempo={vitesse_value} ! autoaudiosink

        
    # Conversion des fichiers :
    
    # mp3 => wav
    conversion_option = dropdown_conversion.value
    if conversion_option == 'mp3_to_wav' and file_name.endswith('.mp3'):
        output_file = f"{file_name.rsplit('.', 1)[0]}.wav"
        !gst-launch-1.0 filesrc location={file_name} ! decodebin ! audioconvert ! wavenc ! filesink location={output_file}
    
    # mp3 => flac
    elif conversion_option == 'mp3_to_flac' and file_name.endswith('.mp3'):
        output_file = f"{file_name.rsplit('.', 1)[0]}.flac"
        !gst-launch-1.0 filesrc location={file_name} ! decodebin ! audioconvert  ! flacenc ! filesink location={output_file}

        
        
    # wav => mp3
    elif conversion_option == 'wav_to_mp3' and file_name.endswith('.wav'):
        output_file = f"{file_name.rsplit('.', 1)[0]}.mp3"
        !gst-launch-1.0 filesrc location={file_name} ! decodebin ! audioconvert  ! lamemp3enc bitrate=192 ! filesink location={output_file}
     
    # wav => flac
    elif conversion_option == 'wav_to_flac' and file_name.endswith('.wav'):
        output_file = f"{file_name.rsplit('.', 1)[0]}.flac"
        !gst-launch-1.0 filesrc location={file_name} ! decodebin ! audioconvert ! flacenc ! filesink location={output_file}

        
        
    # flac => wav
    elif conversion_option == 'flac_to_wav' and file_name.endswith('.flac'):
        output_file = f"{file_name.rsplit('.', 1)[0]}.wav"
        !gst-launch-1.0 filesrc location={file_name} ! decodebin ! audioconvert  ! wavenc ! filesink location={output_file}
   
    # flac => mp3
    elif conversion_option == 'flac_to_mp3' and file_name.endswith('.flac'):
        output_file = f"{file_name.rsplit('.', 1)[0]}.mp3"
        !gst-launch-1.0 filesrc location={file_name} ! decodebin ! audioconvert ! lamemp3enc bitrate=192 ! filesink location={output_file}

        

# Fonction pour activer/désactiver le slider de volume
def checkbox_volume_changement(changement):
    volume_slider.disabled = not checkbox_volume.value# Désactive le slider si la case est décochée, l'active si elle est cochée

# Fonction pour activer/désactiver le slider de vitesse
def checkbox_vitesse_changement(changement):
    vitesse_slider.disabled = not checkbox_vitesse.value
    
    
    

# Toutes les options d'interaction disponible:

# Bouton pour téléverser un fichier
fichier_televerse = widgets.FileUpload(
    accept='.mp3,.wav,.flac', 
    multiple=False, 
    description="Téléverser un fichier",
    button_style="success",
    layout=widgets.Layout(width='30%', height='30px', margin='10px auto', font_weight='bold')
)

# Checkbox pour les différentes fonctionnalitées
checkbox_pochette = widgets.Checkbox(value=False, description='Pochette')
checkbox_metadonnes = widgets.Checkbox(value=False, description='Métadonnées')
checkbox_rps_visu = widgets.Checkbox(value=False, description='Représentation visuelle')
checkbox_rps_spec = widgets.Checkbox(value=False, description='Spectrogramme')
checkbox_volume = widgets.Checkbox(value=False, description='Modifier le volume')  
volume_slider = widgets.FloatSlider(value=1.0, min=0.0, max=10.0, step=0.1, description='Volume', disabled=True)
checkbox_vitesse = widgets.Checkbox(value=False, description='Modifier la vitesse') 
vitesse_slider = widgets.FloatSlider(value=1.0, min=0.5, max=2.0, step=0.1, description='Vitesse', disabled=True)

# Menu déroulant pour les converstions 
dropdown_conversion = widgets.Dropdown(
    options=[
        ('Sélectionner la conversion', None),
        ('MP3 -> WAV', 'mp3_to_wav'),
        ('MP3 -> FLAC', 'mp3_to_flac'),
        ('WAV -> MP3', 'wav_to_mp3'),
        ('WAV -> FLAC', 'wav_to_flac'),
        ('FLAC -> WAV', 'flac_to_wav'),
        ('FLAC -> MP3', 'flac_to_mp3')
    ],
    description='Conversion:'
)

# Bouton pour exécuter ce qu'on a choisit il est lié à la fonction principal
execute_button = widgets.Button(
    description="Exécuter", 
    button_style="primary",
    layout=widgets.Layout(width='30%', height='30px', margin='10px auto', font_weight='bold')
)







        
# Associer les événements aux checkboxes
checkbox_volume.observe(checkbox_volume_changement, names='value')
checkbox_vitesse.observe(checkbox_vitesse_changement, names='value')
execute_button.on_click(execute_bouton)

# Accordion 1 : Niveau Bart
accordion1 = widgets.Accordion(children=[widgets.HBox(
                                             [checkbox_pochette, checkbox_metadonnes, checkbox_rps_visu], layout=widgets.Layout(justify_content='center', width='100%')
                                         )
                                        ]
                              )
accordion1.set_title(0, 'Niveau Bart')

# Accordion 2 : Niveau Lisa
accordion2 = widgets.Accordion(children=[widgets.VBox(
                                             [checkbox_rps_spec, widgets.HBox([checkbox_volume, volume_slider], layout=widgets.Layout(justify_content='center', width='100%')), widgets.HBox([checkbox_vitesse, vitesse_slider], layout=widgets.Layout(justify_content='center', width='100%')),  widgets.HBox([dropdown_conversion], layout=widgets.Layout(justify_content='center', width='100%')) ], layout=widgets.Layout(align_items='center', width='100%')
                                         ) 
                                        ]
                              )
accordion2.set_title(0, 'Niveau Lisa')

# Interface finale cela permet d'ordonner les différents éléments dans l'ordre
widgets_container = widgets.VBox([fichier_televerse,accordion1,accordion2,execute_button])

# Afficher l'interface
widgets_container
